In [2]:
# Install modules
# A '!' in a Jupyter Notebook runs the line in the system's shell, and not in the Python interpreter

# Import necessary libraries
import pandas as pd
import random

# Load dataset
# you can download this dataset from https://huggingface.co/datasets/stepp1/tweet_emotion_intensity/tree/main
data = pd.read_csv('./test.csv')

# Preview the data
print(data.head())

      id                                              tweet    class  \
0  31079  Rear garden alarm activated  at Telford house ...  sadness   
1  11428  @bt_uk appointment booked between 1-6 today, w...     fear   
2  21865  Hoping to hear some serious @DonnieTrumpet ton...     fear   
3  31125  I'm always smiling so that's why I'm always ha...  sadness   
4  30912  @Sirenja_ @JaxOfBo (like, hope i didn't offend...      joy   

   sentiment_intensity class_intensity  labels  
0                  NaN    sadness_NONE      -1  
1                  NaN       fear_NONE      -1  
2                  NaN       fear_NONE      -1  
3                  NaN    sadness_NONE      -1  
4                  NaN        joy_NONE      -1  


In [3]:
import re # Import the `re` module for working with regular expressions

# Function to clean the text
def clean_text(text):
    text = text.lower() # Convert all text to lowercase for uniformity
    text = re.sub(r'http\S+', '', text) # Remove URLs from the text
    text = re.sub(r'<.*?>', '', text) # Remove any HTML tags from the text
    text = re.sub(r'[^\w\s]', '', text) # Remove punctuation, keep only words and spaces
    return text # Return the cleaned text

# Assume `data` is a pandas DataFrame with a column named 'text'
# Apply the cleaning function to each row of the 'text' column
data['cleaned_text'] = data['tweet'].apply(clean_text)

# Print the first 5 rows of the cleaned text to verify the cleaning process
print(data['cleaned_text'].head())

0    rear garden alarm activated  at telford house ...
1    bt_uk appointment booked between 16 today wait...
2    hoping to hear some serious donnietrumpet tonight
3      im always smiling so thats why im always happy 
4    sirenja_ jaxofbo like hope i didnt offend with...
Name: cleaned_text, dtype: object


In [4]:
# Check for missing values in the dataset
print(data.isnull().sum()) # Print the count of missing values for each column

# Option 1: Remove rows with missing data in the 'cleaned_text' column
data = data.dropna(subset=['cleaned_text']) # Drop rows where 'cleaned_text' is NaN (missing)

# Option 2: Fill missing values in 'cleaned_text' with a placeholder
data['cleaned_text'].fillna('unknown', inplace=True) # Replace NaN values in 'cleaned_text' with 'unknown'

id                        0
tweet                     0
class                     0
sentiment_intensity    3142
class_intensity           0
labels                    0
cleaned_text              0
dtype: int64


/var/folders/7t/7p1ldf8d4zj16svxk9h0h34c0000gn/T/ipykernel_5308/3840859666.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['cleaned_text'].fillna('unknown', inplace=True) # Replace NaN values in 'cleaned_text' with 'unknown'


In [5]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the cleaned text
tokens = tokenizer(
    data['cleaned_text'].tolist(), padding=True, truncation=True, max_length=128, return_tensors='pt'
)

print(tokens['input_ids'][:5])  # Preview the first 5 tokenized examples

/Users/phuonghqh/.pyenv/versions/learnai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[  101,  4373,  3871,  8598,  8878,  2012, 10093,  3877,  2160,  2012,
         10114,  2549,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0],
        [  101, 18411,  1035,  2866,  6098, 17414,  2090,  2385,  2651,  4741,
          1999,  2035,  2154,  1998,  6343,  3662,  2039,  2036,  7303,  1037,
          2655,  2067,  1998,  2196,  2288,  2028,  9643,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0],
        [  101,  5327,  2000,  2963,  2070,  3809, 28486, 24456, 22327,  3892,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     

In [8]:
!pip install nltk

import nltk
nltk.download('wordnet')

# Import necessary modules
import random # Random module for generating random numbers and selections
from nltk.corpus import wordnet # NLTK's WordNet corpus for finding synonyms

# Define a function to find and replace a word with a synonym
def synonym_replacement(word):
# Get all synsets (sets of synonyms) for the given word from WordNet
    synonyms = wordnet.synsets(word)

# If the word has synonyms, randomly choose one synonym, otherwise return the original word
    if synonyms:
# Select a random synonym and get the first lemma (word form) of that synonym
        return random.choice(synonyms).lemmas()[0].name()

# If no synonyms are found, return the original word
    return word

# Define a function to augment text by replacing words with synonyms randomly
def augment_text(text):
# Split the input text into individual words
    words = text.split() # Split the input text into individual words

# Replace each word with a synonym with a probability of 20% (random.random() > 0.8)
    augmented_words = [
    synonym_replacement(word) if random.random() > 0.8 else word
# If random condition met, replace
for word in words] # Iterate over each word in the original text

# Join the augmented words back into a single string and return it
    return ' '.join(augmented_words)

# Apply the text augmentation function to the 'cleaned_text' column in a DataFrame
# Create a new column 'augmented_text' containing the augmented version of 'cleaned_text'
data['augmented_text'] = data['cleaned_text'].apply(augment_text)


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/phuonghqh/nltk_data...


In [11]:
import torch # Import PyTorch library
from torch.utils.data import TensorDataset, DataLoader # Import modules to create datasets and data loaders

# Convert tokenized data into PyTorch tensors
# input_ids = tokens['input_ids'] # Extract input IDs from the tokenized data
# attention_masks = tokens['attention_mask'] # Extract attention masks from the tokenized data
input_ids = torch.tensor(tokens['input_ids'])
attention_masks = torch.tensor(tokens['attention_mask'])

# Define a mapping function
def map_sentiment(value):
    if value == "high":
        return 1
    elif value == "medium":
        return 0.5
    elif value == "low":
        return 0
    else:
        return None  # Handle unexpected values, if any

# Apply the function to each item in 'sentiment_intensity'
data['sentiment_intensity'] = data['sentiment_intensity'].apply(map_sentiment)

# Drop any rows where 'sentiment_intensity' is None
data = data.dropna(subset=['sentiment_intensity']).reset_index(drop=True)

# Convert the 'sentiment_intensity' column to a tensor
labels = torch.tensor(data['sentiment_intensity'].tolist())

/var/folders/7t/7p1ldf8d4zj16svxk9h0h34c0000gn/T/ipykernel_5308/2264232787.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(tokens['input_ids'])
/var/folders/7t/7p1ldf8d4zj16svxk9h0h34c0000gn/T/ipykernel_5308/2264232787.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_masks = torch.tensor(tokens['attention_mask'])


In [13]:
from sklearn.model_selection import train_test_split # Import function to split dataset

# First split: 15% for test set, the rest for training/validation
train_val_inputs, test_inputs, train_val_masks, test_masks, train_val_labels, test_labels = train_test_split(
    input_ids, attention_masks, labels, test_size=0.15, random_state=42
)

# Second split: 20% for validation set from remaining data
train_inputs, val_inputs, train_masks, val_masks, train_labels, val_labels = train_test_split(
    train_val_inputs, train_val_masks, train_val_labels, test_size=0.2, random_state=42
)

# Create TensorDataset objects for each set, including attention masks
train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
val_dataset = TensorDataset(val_inputs, val_masks, val_labels)
test_dataset = TensorDataset(test_inputs, test_masks, test_labels)

# Create DataLoader objects
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16)
test_dataloader = DataLoader(test_dataset, batch_size=16)

print("Training, validation, and test sets are prepared with attention masks!")

ValueError: Found input variables with inconsistent numbers of samples: [3142, 3142, 0]